<a href="https://colab.research.google.com/github/Hsieh121/generative_ai/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab 程式碼儲存格
!pip install googlemaps gradio transformers torch bitsandbytes accelerate # accelerate 可能對某些模型有幫助

In [ ]:
import googlemaps
from google.colab import userdata # 用於讀取 Colab Secrets

# 從 Colab Secrets 讀取 API Key
GMAPS_API_KEY = userdata.get('GMAPS_API_KEY') # 假設你將金鑰命名為 GMAPS_API_KEY
gmaps = googlemaps.Client(key=GMAPS_API_KEY)

def get_restaurant_reviews(restaurant_name, location="台北市大安區"):
    try:
        # 1. Text Search (or Find Place) to get place_id
        places_result = gmaps.places(query=f"{restaurant_name} in {location}")

        if not places_result or places_result['status'] != 'OK' or not places_result['results']:
            return None, "找不到餐廳或API查詢失敗。"

        place_id = places_result['results'][0]['place_id'] # 取第一個結果
        place_name_found = places_result['results'][0]['name']
        place_address_found = places_result['results'][0].get('formatted_address', '地址未提供')
        place_rating_found = places_result['results'][0].get('rating', '評分未提供')

        # 2. Place Details to get reviews
        # language='zh-TW' 嘗試獲取繁體中文評論
        details = gmaps.place(place_id=place_id, fields=['name', 'formatted_address', 'rating', 'reviews'], language='zh-TW')

        if details['status'] != 'OK':
            return None, "獲取餐廳詳細資訊失敗。"

        restaurant_info = {
            "name": details['result'].get('name', place_name_found),
            "address": details['result'].get('formatted_address', place_address_found),
            "rating": details['result'].get('rating', place_rating_found),
            "reviews_text": []
        }

        # Places API 通常返回最多 5 條評論
        if 'reviews' in details['result']:
            for review in details['result']['reviews']:
                restaurant_info["reviews_text"].append(f"- {review.get('author_name', '匿名使用者')} (評分: {review.get('rating', 'N/A')}星): {review.get('text', '')}")

        if not restaurant_info["reviews_text"]:
             restaurant_info["reviews_text"].append("暫無足夠的文字評論可供分析。")

        return restaurant_info, None
    except Exception as e:
        return None, f"發生錯誤：{str(e)}"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig # 引入 BitsAndBytesConfig
import torch

# --- 你要載入的模型 ID ---
model_id = "google/gemma-7b-it" # Gemma 7B (需要處理授權和 HuggingFace Token)

model = None  # 先初始化為 None
tokenizer = None # 先初始化為 None

try:
    print(f"準備載入【4-bit 量化】模型: {model_id}")
    print(f"檢查 CUDA 是否可用: {torch.cuda.is_available()}") # 確認 CUDA 環境

    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # --- 設定 4-bit 量化配置 ---
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",                # 指定量化類型，"nf4" (NormalFloat4) 是常用的選擇
        bnb_4bit_compute_dtype=torch.bfloat16,    # 設定計算時使用的資料類型，bfloat16 可提升效能
                                                  # 如果你的 GPU 對 bfloat16 支援不佳，可以改用 torch.float16
        bnb_4bit_use_double_quant=False           # 是否使用雙重量化，通常設為 False 即可
    )

    # --- 載入量化模型 ---
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config, # <--- 傳入量化設定
        device_map="auto"                        # <--- 讓 Transformers 自動分配到 GPU (如果可用)
    )
    print(f"【4-bit 量化】模型 {model_id} 已載入到設備: {model.device}")
    if hasattr(model, 'get_memory_footprint'): # 檢查模型物件是否有 get_memory_footprint 方法
        print(f"模型記憶體佔用 (近似): {model.get_memory_footprint() / 1024**3:.2f} GB") # 量化後應遠小於 14GB

except Exception as e:
    print(f"載入【4-bit 量化】模型 {model_id} 時發生錯誤 (可能是資源不足、需要授權、模型名稱錯誤或 bitsandbytes 未正確安裝/版本過舊): {e}")
    import traceback
    traceback.print_exc() # 打印詳細的錯誤堆疊

In [ ]:
def generate_llm_output(prompt_text):
    if not model or not tokenizer:
        return "LLM 模型未能成功載入，無法生成文字。"
    try:
        inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device) # 將輸入移至模型所在的設備
        outputs = model.generate(**inputs, max_new_tokens=250, pad_token_id=tokenizer.eos_token_id) # pad_token_id 避免警告
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        #print(f"DEBUG generate_llm_output: Full decoded result before processing: {result}") # <--- 新增
        # Gemma 模型的回應可能包含原始 prompt，需要處理
        # 簡單的處理方式是移除 prompt_text 的部分
        if result.startswith(prompt_text):
             return result[len(prompt_text):].strip()
        return result.strip() # 確保移除前後空白
    except Exception as e:
        return f"LLM 生成時發生錯誤：{str(e)}"

def analyze_reviews_with_llm(reviews_list):
    reviews_str = "\n".join(reviews_list)

    # 1. 評論總結
    prompt_summary = f"""作為一個客觀的美食評論分析員，請仔細閱讀以下顧客評論。
根據這些評論，請用繁體中文為這家餐廳寫一段精簡的整體評價總結，字數約200字（繁體中文）。
總結應涵蓋顧客提及的主要正面和負面體驗，如果沒有明顯的負面體驗，則著重描述正面評價。
請不要自行編造評論中未提及的內容，並回答評價內容部分就好，其他回應不需要，並翻譯成繁體中文。

顧客評論如下：
{reviews_str}

專業評論總結："""
    summary = generate_llm_output(prompt_summary)

    '''# 2. 生成 AI 食記片段 (假設模仿一位活潑的美食部落客)
    prompt_creative_review = f"假設你是一位活潑的美食部落客，請根據以下顧客評論，為這家餐廳寫一段生動有趣的食記片段（約80-120字，繁體中文），要包含一些情緒和感受：\n\n{reviews_str}\n\n食記片段："
    creative_review = generate_llm_output(prompt_creative_review)

    # 3. 提取推薦菜色 (也可以用更簡單的關鍵字提取)
    prompt_dishes = f"從以下顧客評論中，提取出顧客們推薦的菜色名稱 (如果有的話，請列點說明，繁體中文)：\n\n{reviews_str}\n\n推薦菜色："
    dishes = generate_llm_output(prompt_dishes)'''

    return summary

In [ ]:
import gradio as gr

# 假設你的 get_restaurant_reviews, analyze_reviews_with_llm (現在只處理總結),
# AutoTokenizer, AutoModelForCausalLM, model, tokenizer 等都已經在前面定義好了。

# 我假設 analyze_reviews_with_llm 現在經過修改，
# 或者你只使用它回傳的第一個值作為總結。
# 例如，如果 analyze_reviews_with_llm 原本返回 (summary, creative, dishes)，
# 你可以這樣調用：
# llm_summary_text, _, _ = analyze_reviews_with_llm(restaurant_data['reviews_text'])
# 或者，更好的方式是修改 analyze_reviews_with_llm 讓它只計算並返回 summary_text。
# 這裡，我將假設 analyze_reviews_with_llm 只返回一個總結字串。

def gradio_interface(restaurant_name_input):
    if not restaurant_name_input.strip():
        # 修改：錯誤時返回 3 個值
        return "錯誤：請輸入餐廳名稱。", "", ""

    restaurant_data, error_msg = get_restaurant_reviews(restaurant_name_input)

    if error_msg:
        # 修改：錯誤時返回 3 個值
        return f"錯誤：{error_msg}", "", ""

    if not restaurant_data:
        # 修改：錯誤時返回 3 個值
        return "錯誤：未能獲取餐廳資訊。", "", ""

    # 顯示基本資訊
    info_md = f"""
    ### {restaurant_data['name']}
    **地址：** {restaurant_data['address']}
    **Google 總體評分：** {restaurant_data['rating']} 星
    """

    raw_reviews_display = "\n".join(restaurant_data['reviews_text']) if restaurant_data['reviews_text'] else "暫無評論可顯示。"

    llm_summary_text = "" # 初始化
    # 使用 LLM 處理評論
    if not model or not tokenizer: # 檢查模型是否成功載入
        llm_summary_text = "LLM 模型未能成功載入，無法進行分析。"
    elif not restaurant_data['reviews_text'] or \
         (restaurant_data['reviews_text'] and restaurant_data['reviews_text'][0] == "暫無足夠的文字評論可供分析。"):
        llm_summary_text = "沒有足夠的評論內容可供 LLM 分析。"
    else:
        # 假設 analyze_reviews_with_llm 現在只負責生成並返回總結字串
        # 如果它之前返回多個值，你需要像這樣調整：
        # temp_summary, _, _ = analyze_reviews_with_llm(restaurant_data['reviews_text'])
        # llm_summary_text = temp_summary
        # 為了簡潔，我們假設 analyze_reviews_with_llm 已被修改為只返回總結
        llm_summary_text = analyze_reviews_with_llm(restaurant_data['reviews_text'])

    # 修改：只返回 3 個值
    return info_md, raw_reviews_display, llm_summary_text

# 建立 Gradio 介面
with gr.Blocks(title="AI 美食評論家") as iface:
    gr.Markdown("# 🍜 AI 美食評論家 (台北市大安區)")
    gr.Markdown("輸入大安區的餐廳名稱，AI 將從 Google Maps 獲取評論，並為您生成評論摘要！") # 修改了描述

    restaurant_input = gr.Textbox(label="請輸入餐廳名稱 (預設搜尋台北市大安區)")
    submit_button = gr.Button("提交，讓 AI 分析！")

    gr.Markdown("---")
    output_info = gr.Markdown(label="餐廳基本資訊")
    output_raw_reviews = gr.Textbox(label="Google Maps 原始評論摘錄 (最多5條)", lines=7, interactive=False)
    output_summary = gr.Textbox(label="AI 評論總結", lines=4, interactive=False)

    # 以下兩個 Textbox 已被刪除
    # output_creative = gr.Textbox(label="AI 風格食記片段", lines=6, interactive=False)
    # output_dishes = gr.Textbox(label="AI 提取推薦菜色", lines=3, interactive=False)

    submit_button.click(
        gradio_interface,
        inputs=[restaurant_input],
        # 修改：outputs 列表只包含 3 個元件
        outputs=[output_info, output_raw_reviews, output_summary]
    )

iface.launch(debug=True)